In [1]:
import pandas as pd
import json
from collections import OrderedDict
from itertools import groupby
from datetime import datetime
from pymongo import MongoClient

In [2]:
# Connect
client = MongoClient("mongodb://mongo:27017")

# Connect to `inspector_restaurant` database
db = client['inspector_restaurant']

In [3]:
# NOTE - change the `nrows=None` argument to a smaller number for tesitng
# i.e. `nrows=100` will only load the first 100 rows
# df = pd.read_csv('Food_Service_Establishment_Inspections__Beginning_2005__ACTIVE_.csv', nrows=1000)
df = pd.read_csv('nyc_restaurants.csv')

In [4]:
# df.head()

In [5]:
results = dict()
violationLookup = dict()
errorCount = 0

print('Processing CSV...')

# Iterates over each row in the dataframe
for row in df.iterrows():
    index, data = row
    
#     if isinstance(data['ADDRESS'], float):
#         # print(data)
#         errorCount += 1
#         continue
#     elif ',  ' not in data['ADDRESS']:
#         # print(data)
#         errorCount += 1
#         continue
    
    # Defines defaultRestaurant object
    defaultRestaurant = dict({
        'facility_id': int(data['CAMIS']),
        'facility': str(data['DBA']).strip().title().replace("'S","'s"),
        'operation_name': str(data['DBA']).strip().title().replace("'S","'s"),
        'description': data['CUISINE DESCRIPTION'],
        'address': dict({
            'street': str(data['BUILDING']) + ' ' + str(data['STREET']).strip().title(),
            'city': 'New York',
            'state': 'NY',
            'zipcode': str(data['ZIPCODE'])
        }),
        'inspections': []
    })
    
    # Gets the parent restaurant, or returns defaultRestaurant
    restaurant = results.get(data['CAMIS'], defaultRestaurant)

    # Defines the default parent inspection for this violation 
    inspection = dict({
        #'date': datetime.strptime(date, "%m/%d/%y"),
        'date': data['INSPECTION DATE'],
        'type': data['INSPECTION TYPE'],
        'comment': '',
        'violations': []
    })
    
    # Gets the parent inspection, or returns defaultInspection
    # inspection = restaurant['inspections'].get(data['DATE OF INSPECTION'], defaultInspection)
    hasParent = False
    for insp in restaurant['inspections']:
        if (insp['date'] == inspection['date']):
            inspection = insp
            hasParent = True

    # Adds the individual violation to the inspection record
    inspection['violations'].append(data['VIOLATION CODE'])

    # Tracks the individual violation
    violationLookup[data['VIOLATION CODE']] = dict({
        "desc": data['VIOLATION DESCRIPTION'],
        "critical": data['CRITICAL FLAG']
    })
        
    # Adds the inspection to the parent restaurant
    # restaurant['inspections'][data['DATE OF INSPECTION']] = inspection
    if (not hasParent):
        restaurant['inspections'].append(inspection)
    
    # Adds the restaurant to our complete list
    results[data['CAMIS']] = restaurant
    
print('Done processing')

Processing CSV...
Done processing


In [6]:
errorCount

0

In [7]:
# Iterates over each Restaurant and inserts it into Mongo
for k, v in results.items():
    db['restaurants'].insert_one(v)
#     print(json.dumps(v, indent=2)) 

# Logs..
# print('Wrote Restaurants to MongoDb...')

In [8]:
# Writes the Restaurants to JSON
# file = open('restaurants.json','w') 
# file.write(json.dumps(output, indent=2)) 
# file.close()
# print(json.dumps(violationLookup, indent=2))

In [9]:
# Iterates over each Violation and inserts it into Mongo
for v in violationLookup.items():
    db['violations'].insert_one(dict({ 'vid': v[0], 'desc': v[1]['desc'], 'critical': v[1]['critical'] == 'Critical'}))
    # print(json.dumps(dict({ 'vid': v[0], 'desc': v[1]['desc'], 'critical': v[1]['critical'] == 'Critical'}), indent=2))

# Logs..
print('Wrote Violations to MongoDb...')

Wrote Violations to MongoDb...
